In [7]:
import numpy as np
import tensorflow as tf
from tensorflow.python.estimator import run_config as run_config_lib
from tensorflow.python.platform import tf_logging as logging
from tensorflow.examples.tutorials.mnist import input_data as input_data
import math
print("Tensorflow version " + tf.__version__)
logging.set_verbosity(logging.INFO)

mnist = input_data.read_data_sets("MNIST_data/", one_hot=False, reshape=True, validation_size=0)

Tensorflow version 1.2.1
Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [15]:
def conv_model_loss( Ylogits, Y_, mode):
    if mode == tf.estimator.ModeKeys.TRAIN or mode == tf.estimator.ModeKeys.EVAL:
        return tf.reduce_mean( tf.losses.softmax_cross_entropy( tf.one_hot( Y_,10), Ylogits)) * 100
    return None

def conv_model_train_op( loss, mode):
    if mode == tf.estimator.ModeKeys.TRAIN:
        return tf.contrib.layers.optimize_loss( loss, tf.train.get_global_step( ), 
                                               learning_rate=0.003, 
                                               optimizer="Adam",
                                               learning_rate_decay_fn = lambda lr, step: 0.0001 + tf.train.exponential_decay( lr, step, -2000, math.e))
    return None

def conv_model_eval_metrics( classes, Y_, mode):
    if mode == tf.estimator.ModeKeys.TRAIN or mode == tf.estimator.ModeKeys.EVAL:
        return {'accuracy': tf.metrics.accuracy( classes, Y_)}
    return None
    
def conv_model( features, labels, mode):
    X = tf.reshape( features, [-1, 28, 28, 1])
    Y_ = labels
    
    biasInit = tf.constant_initializer( 0.1, dtype=tf.float32)
    Y1 = tf.layers.conv2d( X,  filters=6,  kernel_size=[6, 6], padding="same", activation=tf.nn.relu, bias_initializer=biasInit)
    Y2 = tf.layers.conv2d( Y1, filters=12, kernel_size=[5, 5], padding="same", strides=2, activation=tf.nn.relu, bias_initializer=biasInit)
    Y3 = tf.layers.conv2d( Y2, filters=24, kernel_size=[4, 4], padding="same", strides=2, activation=tf.nn.relu, bias_initializer=biasInit)
    
    Y4 = tf.reshape( Y3, [-1, 24*7*7])
    Y5 = tf.layers.dense( Y4, 200, activation=tf.nn.relu, bias_initializer=biasInit)
    Y5d = tf.layers.dropout( Y5, rate=0.25, training=mode==tf.estimator.ModeKeys.TRAIN)
    Ylogits = tf.layers.dense( Y5d, 10)
    
    predict = tf.nn.softmax( Ylogits)
    classes = tf.cast( tf.argmax( predict, 1), tf.uint8)

    loss = conv_model_loss( Ylogits, Y_, mode)
    train_op = conv_model_train_op( loss, mode)
    eval_metrics = conv_model_eval_metrics( classes, Y_, mode)

    return tf.estimator.EstimatorSpec( 
        mode=mode,
        predictions={"predictions": predict, "classes": classes},
        loss=loss,
        train_op=train_op,
        eval_metric_ops=eval_metrics
    )

In [13]:
class CustomRunConfig(run_config_lib.RunConfig):
    @property
    def save_checkpoints_secs(self): return None
    @property
    def save_checkpoints_steps(self): return 1000
    @property
    def tf_random_seed(self): return 0
    
estimator = tf.estimator.Estimator(model_fn=conv_model, model_dir="checkpoints", config=CustomRunConfig())

def train_data_input_fn():
    return tf.train.shuffle_batch([tf.constant(mnist.train.images), tf.constant(mnist.train.labels)],
                                  batch_size=100, capacity=1100, min_after_dequeue=1000, enqueue_many=True)

def eval_data_input_fn():
    return tf.constant(mnist.test.images), tf.constant(mnist.test.labels)

def predict_input_fn():
    return tf.constant(test_digits)

INFO:tensorflow:Using config: {'_keep_checkpoint_every_n_hours': 10000, '_save_summary_steps': 100, '_tf_random_seed': 1, '_save_checkpoints_steps': None, '_model_dir': 'checkpoints', '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5}


In [14]:
estimator.train( input_fn = train_data_input_fn, steps=1000)
estimator.evaluate( input_fn = eval_data_input_fn, steps=1)
digits = estimator.predict( input_fn = predict_input_fn)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into checkpoints/model.ckpt.
INFO:tensorflow:loss = 231.343, step = 1
INFO:tensorflow:global_step/sec: 114.158
INFO:tensorflow:loss = 31.2677, step = 101 (0.878 sec)
INFO:tensorflow:global_step/sec: 119.032
INFO:tensorflow:loss = 6.17775, step = 201 (0.840 sec)
INFO:tensorflow:global_step/sec: 126.975
INFO:tensorflow:loss = 11.3744, step = 301 (0.788 sec)
INFO:tensorflow:global_step/sec: 121.902
INFO:tensorflow:loss = 12.8321, step = 401 (0.820 sec)
INFO:tensorflow:global_step/sec: 133.591
INFO:tensorflow:loss = 4.52029, step = 501 (0.748 sec)
INFO:tensorflow:global_step/sec: 116.593
INFO:tensorflow:loss = 4.30247, step = 601 (0.859 sec)
INFO:tensorflow:global_step/sec: 130.081
INFO:tensorflow:loss = 5.76733, step = 701 (0.767 sec)
INFO:tensorflow:global_step/sec: 142.588
INFO:tensorflow:loss = 6.01962, step = 801 (0.703 sec)
INFO:tensorflow:global_step/sec: 124.477
INFO:tensorflow:loss = 4.77663, step